# ⚖️ Agentic Legal Assistant Demo

This notebook demonstrates how to use an agentic RAG pipeline built with LangGraph + LangChain to ask questions about a legal contract PDF.

---

### 🟩 **Install**

In [30]:
#!pip install langchain langgraph langchain-openai langchain-huggingface chromadb ipywidgets tqdm

---

### 🟩 **Load PDF**

In [18]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("contracts/sample_contract.pdf")
docs = loader.load()

print(f"Loaded {len(docs)} pages")
print(docs[0].page_content[:1000])

Loaded 2 pages
SAMPLE CONTRACT  
 
SERVICE AGREEMENT 
This Service Agreement ("Agreement") is entered into on July 7, 2025, by and between: 
Client: ABC Corporation 
Contractor: Jane Doe Consulting LLC 
 
1. Services Provided 
Contractor shall provide data analytics and reporting services as requested by Client. 
 
2. Term 
This Agreement begins on the effective date and continues until either party provides 30 days 
written notice of termination. 
 
3. Termination 
This contract may be terminated: 
• By either party for any reason with 30 days’ notice. 
• Immediately by either party in the event of material breach. 
 
4. Payment 
Client agrees to pay Contractor $10,000 per month, invoiced on the 1st of each month. 
 
5. Confidentiality 
All data and materials provided by Client shall remain confidential.


---

### 🟩 Embed & Index**

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(docs, embedding=embedding)
retriever = vectorstore.as_retriever()


---

### 🟩 **Setup LLM and RAG Chain**

In [23]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(temperature=0)
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever)


---

### 🟩 **Ask a Question**

In [25]:
chat_history = []

question = "Does this contract include a termination clause?"
result = qa_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.append((question, result["answer"]))

print("Answer:", result["answer"])

Answer: Yes, this contract does include a termination clause. It states that the contract may be terminated by either party for any reason with 30 days' notice, or immediately by either party in the event of a material breach.


---

### 🟩 **Save Q\&A Log**

In [32]:
import json
from datetime import datetime

qa_log = [{"timestamp": datetime.now().isoformat(), "question": q, "answer": a} for q, a in chat_history]
with open("outputs/qa_log.jsonl", "w") as f:
    for entry in qa_log:
        f.write(json.dumps(entry) + "\n")